In [26]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
%matplotlib notebook
tf.disable_v2_behavior() 

In [27]:
train = pd.read_csv(r"train.csv")
test  = pd.read_csv(r"test.csv")

In [28]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
y = train["label"]

In [30]:
x = train.drop("label",axis=1) 

In [31]:
x.shape

(42000, 784)

In [32]:
from sklearn.model_selection import train_test_split 

In [33]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)

In [34]:
print(x_train.shape)
x_train = np.array(x_train)
x1 = x_train.reshape(-1,28,28,1)
print(x1.shape)

(33600, 784)
(33600, 28, 28, 1)


In [35]:
y_train.shape
y_train = pd.get_dummies(y_train)


In [36]:
y_train.shape

(33600, 10)

In [37]:
#creating variables or parameters for our neural net
n_input_pixels = 784

shape1 = 28
shape2 = 28

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k      = 5
conv2_k      = 5
max_pool1_k  = 2
max_pool2_k  = 2

n_input_channels = 1
n_hidden_1  = 256
n_hidden_2  = 256
n_classes       = 10

n_hidden_input = (shape1//(max_pool1_k*max_pool2_k)) * (shape2//(max_pool1_k*max_pool2_k)) *n_conv2

#creating the weight dictionary 
#check the copy of how the sizes have been defined
weights = {
    'w1'  : tf.Variable(tf.random.normal([conv1_k,conv1_k,n_input_channels,n_conv1])),
    'w2'  : tf.Variable(tf.random.normal([conv2_k,conv2_k,n_input_channels,n_conv2])),
    'h1'  : tf.Variable(tf.random.normal([n_hidden_input,n_hidden_1])),
    'h2'  : tf.Variable(tf.random.normal([n_hidden_1,n_hidden_2])),
    'out' : tf.Variable(tf.random.normal([n_hidden_2,n_classes]))
}

#creating the bias dictionary 
#check the copy of how the sizes have been defined
baises = {
    'w1' : tf.Variable(tf.random.normal([n_conv1])),
    'w2' : tf.Variable(tf.random.normal([n_conv2])),
    'h1' : tf.Variable(tf.random.normal([n_hidden_1])),
    'h2' : tf.Variable(tf.random.normal([n_hidden_2])),
    'out' : tf.Variable(tf.random.normal([n_classes]))
}

In [38]:
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [39]:
#this is the forward propagation algo.
def forward_propagation(x,weights,biases) :
    cnn_layer1 = conv(x,weights['w1'],biases['w1'],stride_conv1)
    cnn_layer1 = maxpooling(cnn_layer1,max_pool1_k)
    
    cnn_layer2 = conv(cnn_layer1,weights['w2'],biases['w2'],stride_conv2)
    cnn_layer2 = maxpooling(cnn_layer2,max_pool2_k)
    x1 = tf.reshape(cnn_layer2,shape=[-1,n_hidden_input])
    
    in_layer1  = tf.add(tf.matmul(x1,weights['h1']),baises['h1'])
    out_layer1 = tf.nn.relu(in_layer1)
    
    in_layer2  = tf.add(tf.matmul(out_layer1,weights['h2']),baises['h2'])
    out_layer2 = tf.nn.relu(in_layer2)
    
    output = tf.add(tf.matmul(out_layer2,weights['out']),baises['out'])
    return output

In [40]:
#creating placehloders for x and y for traing and testing data
xp    = tf.placeholder("float",[None,shape1,shape2,n_input_channels])
yp    = tf.placeholder(tf.int32,[None,n_classes])
#this is forward propagation function call
pred = forward_propagation(xp,weights,baises)

In [41]:
#this is one of the cost function of tensorflow 
#the cost function used here is cross entropy function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels = yp))

In [42]:
#creating session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [43]:
#this is the optimizer section 
#we have used adams optimizer here
#the last line of the cell removed the error.
#the optimizer optimizes the values of the cost only once 
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)
sess.run(tf.global_variables_initializer())

In [44]:
for i in range(20):
    c,_ = sess.run([cost,optimize],feed_dict={xp:x1,yp:y_train})
    print(c)

UnimplementedError: Fused conv implementation does not support grouped convolutions for now.
	 [[node Relu_5 (defined at <ipython-input-38-a29543ce9a05>:4) ]]

Errors may have originated from an input operation.
Input Source operations connected to node Relu_5:
 BiasAdd_3 (defined at <ipython-input-38-a29543ce9a05>:3)

Original stack trace for 'Relu_5':
  File "E:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "E:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "E:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "E:\Anaconda\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "E:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "E:\Anaconda\lib\site-packages\tornado\platform\asyncio.py", line 149, in start
    self.asyncio_loop.run_forever()
  Fil

In [57]:
#this is the testing part of the algorithm
#here argmax is a important function
#all these tensorflow functions are important
#argmax gives the index of the max value index
predictions = tf.argmax(pred,1)
correct_labels = tf.argmax(yp,1)
#tf.equal calculates makes a array with values : 
#true for correct answer
#and false for incorrect
correct_predictions = tf.equal(predictions,correct_labels)

In [58]:
y_test = pd.get_dummies(y_test)
y_test.head()

,0,1,2,3,4,5,6,7,8,9
16275,0,0,0,1,0,0,0,0,0,0
19204,0,0,0,0,0,0,1,0,0,0
18518,0,0,0,0,0,0,0,0,0,1
25780,0,0,0,0,0,1,0,0,0,0
16228,0,0,0,0,0,0,1,0,0,0


In [59]:
y_test.shape

(8400, 10)

In [60]:
x_test.shape

(8400, 784)

In [69]:
#this is the final step where we have called the algo on the testing data
predictions_,labels,result = sess.run([predictions,correct_labels,correct_predictions],feed_dict={xp:x_test , yp:y_test})
predictions_,labels,result

(array([3, 6, 9, ..., 2, 7, 2], dtype=int64),
 array([3, 6, 9, ..., 2, 7, 2], dtype=int64),
 array([ True,  True,  True, ...,  True,  True,  True]))

In [72]:
result.sum()/8400

0.9235714285714286